In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.


In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
msmt_test = session.query(Measurement).order_by(Measurement.date.desc()).first()
last_test = pd.to_datetime(msmt_test.date)
first_test = last_test - timedelta(days=365)
first_date = dt.date(first_test.year, first_test.month, first_test.day)
last_date = dt.date(last_test.year, last_test.month, last_test.day)

# Perform a query to retrieve the data and precipitation scores
msmt_year = session.query(Measurement.date,Measurement.prcp).\
    filter(Measurement.date >= first_date).\
    order_by(Measurement.date.asc()).\
    all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
precip_year_df = pd.DataFrame(msmt_year,columns=['Date','Precipitation'])
precip_year_df = precip_year_df.set_index('Date')
precip_year_df = precip_year_df.fillna(0)

# Sort the dataframe by date
precip_year_df.sort_values(by='Date')
precip_year_df.head()

# Use Pandas Plotting with Matplotlib to plot the data
plt.style.use('fivethirtyeight')
precip_year_df.plot(figsize=(9,6))
plt.xticks([])
plt.legend(['Precipitation'])
plt.xlabel('Date')
plt.ylabel('Precipitation')
plt.title('Last 12 Months of Precipitation Data')
plt.tight_layout()
plt.savefig('Images/prcp12months.png')
plt.show()

NameError: name 'timedelta' is not defined

In [11]:
# Use Pandas to calculate the summary statistics for the precipitation data
precip_year_df.describe()

NameError: name 'precip_year_df' is not defined

# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number of stations in the dataset
num_stations = session.query(Station).count()
print(f'The total number of stations in this dataset is {num_stations}')

The total number of stations in this dataset is 9


In [13]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_count = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()#.distinct(Station.station).count()#group_by(Station.station).all()
station_count = {station: count for station, count in station_count}
print('Stations and Counts:')
print('--------------------')
for station,count in station_count.items():
    print(station,count)
print('====================')

Stations and Counts:
--------------------
USC00519281 2772
USC00519397 2724
USC00513117 2709
USC00519523 2669
USC00516128 2612
USC00514830 2202
USC00511918 1979
USC00517948 1372
USC00518838 511


In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
top_station_temps = session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)).filter(Measurement.station == top_station).all()
print(top_station_temps)
top_station_low, top_station_high, top_station_avg = top_station_temps[0]

print(f'\nFor the most active station ({top_station}):')
print(f'Lowest temperature recorded: {top_station_low}')
print(f'Highest temperature recorded: {top_station_high}')
print(f'Average temperature recorded: {top_station_avg}')

NameError: name 'top_station' is not defined

In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_year = session.query(Measurement.date,Measurement.tobs).filter(Measurement.date >= first_date).filter(Measurement.date <= last_date).filter(Measurement.station == top_station).all()

NameError: name 'first_date' is not defined

# Close Session

In [16]:
# Close Session
session.close()